## Оценка дополнительная с помощью других метрик

### Подключаем датасет

In [1]:
# пути
path3 = '../all_ratings/eval_rag_step_retrieval/html_split/evaluete_rag_compressor_rerank.csv'

In [2]:
import pandas as pd

df = pd.read_csv(path3)
df.head()

,question,contexts,ground_truth,answer,faithfulness,answer_relevancy,context_precision,context_relevancy,answer_similarity,bleu_score,cos-sim-TF-IDF,sim-spacy,cos-sim-BertModel
0,Можно ли не расписываться на карте?,"[""['После получения карты поставьте свою подпи...",При отсутствии подписи карта может быть не при...,"Да, в некоторых случаях вам может быть отказан...",0.0,0.000000,1.0,1.0,0.844934,0.0,0.094760,0.748733,0.983779
1,Мой друг попросил меня дать свою карту для опл...,"[""['Никогда не принимайте рекомендации, советы...","Не передавайте карту другим лицам, в т.ч. родс...","Не люблю менять тему разговора, но вот сейчас ...",NaN,0.000000,1.0,1.0,0.763317,0.0,0.056080,0.576816,0.946034
2,Безопасно ли хранить пинкод в заметках в телеф...,"[""['Категорически запрещается записывать ПИН-к...","Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно хранить пинкод в заметках в ...",1.0,0.000000,1.0,1.0,0.876361,0.0,0.056122,0.576820,0.967247
3,Я потеряла карту,"[""[' Вы потеряли карту или ее украли\\nПрежде ...","Если вы не уверены, что карту украли, не спеши...","Если вы потеряли карту, вам следует немедленно...",NaN,0.870639,1.0,0.2,0.908953,0.0,0.188794,0.906112,0.981569
4,Почему при использовании карты важно не распис...,"[""['Проверяйте правильность суммы, указанной н...",Важно проверять правильность суммы перед подпи...,"Что-то в вашем вопросе меня смущает. Может, по...",NaN,0.000000,1.0,1.0,0.758478,0.0,0.034490,0.730067,0.961646


### Оценка с косинусным сходством 


Оба метода используют косинусное сходство для оценки схожести между текстами, однако различаются в подходе к векторизации текста.

**Первый метод**:
- Использует метод TF-IDF для векторизации текстов.
- Тексты преобразуются в вектора TF-IDF, где каждый элемент вектора представляет собой вес термина в тексте, отражающий его важность в контексте корпуса документов.
- Вычисляется косинусное сходство между векторами TF-IDF.

**Второй метод**:
- Использует предварительно обученную модель BERT для векторизации текстов.
- Тексты представляются в виде последовательности токенов и затем подаются на вход модели BERT.
- Векторные представления получаются из последнего скрытого состояния модели BERT.
- Вычисляется косинусное сходство между векторами представлений текстов.

Оба метода в конечном итоге позволяют оценить схожесть между текстами, но подходы к векторизации их различаются. Первый метод использует контекстуальные представления слов с помощью модели BERT, что может учитывать семантические и контекстуальные аспекты текста, в то время как второй метод использует TF-IDF, который учитывает частоту встречаемости слов в тексте и в корпусе документов, но не учитывает контекст или семантику.


In [59]:
# 1 метод cos-sim-TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity_TF_IDF(row):
    documents = [row['answer'], row['ground_truth']]
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(documents)
    similarity = cosine_similarity(matrix)
    return similarity[0][1]


# Применение функции к DataFrame
df['cos-sim-TF-IDF'] = df.apply(calculate_cosine_similarity_TF_IDF, axis=1)

### Оценка BERTScorer - только на Python 3.10.4

In [60]:
# 2 метод cos-sim-BertModel
# Оценка с BertModel - только на Python 3.10.4
from transformers import BertTokenizer, BertModel
import numpy as np


# Загрузка предварительно обученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Определение функции для вычисления косинусного сходства с использованием модели BERT
def calculate_cosine_similarity_BertModel(row):
    text1 = row['answer']
    text2 = row['ground_truth']
    
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
    
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)
    
    embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()
    
    similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))
    return similarity[0][0]

# Применение функции к DataFrame
df['cos-sim-BertModel'] = df.apply(calculate_cosine_similarity_BertModel, axis=1)

In [61]:
df.tail()

,question,contexts,ground_truth,answer,faithfulness,answer_relevancy,context_precision,context_relevancy,answer_similarity,bleu_score,cos-sim-TF-IDF,sim-spacy,cos-sim-BertModel
53,Могу я отказаться от получения уведомлений от ...,"[""['Клиенты имеют право отказаться от получени...",Клиенты могут отказаться от получения определе...,"Да, клиент может отказаться от получения уведо...",NaN,0.942669,1.0,1.000000,0.945204,0.0,0.335067,0.896644,0.990043
54,Как я могу увеличить лимит по своей кредитной ...,"[""['Для увеличения кредитного лимита клиенты д...","Чтобы увеличить кредитный лимит, клиентам необ...",Для увеличения лимита по своей кредитной карте...,0.666667,0.983427,1.0,1.000000,0.936293,0.0,0.130649,0.884520,0.985550
55,"Каковы условия отмены банковской операции, нап...","[""['Клиенты могут отменить определенные банков...",Условия отмены банковской операции зависят от ...,"Что-то в вашем вопросе меня смущает. Может, по...",NaN,0.000000,1.0,1.000000,0.754078,0.0,0.000000,0.724197,0.933387
56,Какие услуги доступны клиентам через Интернет-...,"[""['Через Интернет-банк клиенты могут управлят...",Интернет-банк предоставляет клиентам возможнос...,Через Интернет-банк клиентам доступны следующи...,1.000000,1.000000,1.0,1.000000,0.927144,0.0,0.135068,0.920870,0.982249
57,"Я не пользовалась кредиткой год, но у меня там...","['[\'""8. Расторжение Договора кредитной карты\...",Согласно пунуту 8.6 - вы не можете вернуть ваш...,"Вы можете вернуть свои 800 рублей, которые ост...",1.000000,0.000000,1.0,0.076923,0.880208,0.0,0.112980,0.782007,0.976573


In [62]:
df.to_csv(path3, index=False)